In [1]:
!wget https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/dl20/topics.dl20.txt
!wget https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/dl20/qrels.dl20-passage.txt
!wget https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/dl20/run.bm25.dl20.txt
!wget https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/collection.tsv

--2024-05-27 23:44:10--  https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/dl20/topics.dl20.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 2607:f8b0:4007:80f::201b, 2607:f8b0:4007:810::201b, 2607:f8b0:4007:803::201b, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2607:f8b0:4007:80f::201b|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2276 (2.2K) [text/plain]
Saving to: ‘topics.dl20.txt’

topics.dl20.txt     100%[===================>]   2.22K  --.-KB/s    in 0s      

2024-05-27 23:44:10 (32.4 MB/s) - ‘topics.dl20.txt’ saved [2276/2276]

--2024-05-27 23:44:10--  https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/dl20/qrels.dl20-passage.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 2607:f8b0:4007:80f::201b, 2607:f8b0:4007:810::201b, 2607:f8b0:4007:811::201b, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2607:f8b0:4007:80f::201b|:443... connected.
HTTP request sent, awaiting

In [2]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv

--2024-05-27 23:45:21--  https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv
Resolving storage.googleapis.com (storage.googleapis.com)... 2607:f8b0:4007:803::201b, 2607:f8b0:4007:810::201b, 2607:f8b0:4007:80f::201b, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2607:f8b0:4007:803::201b|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8076179 (7.7M) [text/tab-separated-values]
Saving to: ‘msmarco_triples.train.tiny.tsv’

msmarco_triples.tra 100%[===================>]   7.70M  49.3MB/s    in 0.2s    

2024-05-27 23:45:22 (49.3 MB/s) - ‘msmarco_triples.train.tiny.tsv’ saved [8076179/8076179]



# Fix seed

In [ ]:
import random
import torch
import torch.nn.functional as F
import numpy as np

random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

In [3]:
!nvidia-smi

Mon May 27 23:52:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.113.01             Driver Version: 535.113.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:01:00.0 Off |                    0 |
| N/A   33C    P0              56W / 300W |      7MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]= "1"

In [7]:
max_train = 20_000
max_valid = 2_000

def serialize_query_passage(query, passage):
    return f'Query: {query} Passage: {passage}'


def load_examples(path):
    x = []
    y = []
    with open(path) as f:
        for line in f:
            query, positive_passage, negative_passage = line.strip().split('\t')
            x.append(serialize_query_passage(query, positive_passage))
            y.append(True)
            x.append(serialize_query_passage(query, negative_passage))
            y.append(False)
    return x, y

x_train, y_train = load_examples('msmarco_triples.train.tiny.tsv')

print(f"Loaded dataset size: x-{len(x_train)}, y-{len(y_train)}")

x_valid = x_train[-max_valid:]
y_valid = y_train[-max_valid:]
x_train = x_train[:max_train]
y_train = y_train[:max_train]

print(len(x_train), 'training examples.')
print(len(x_valid), 'valid examples.')

print('\nFirst 3 training examples:')
for x, y in zip(x_train[:3], y_train[:3]):
    print(f'Label: {y} Input: "{x}"')

print('\n' + 100 * '-' + '\n\nFirst 3 validation examples:')
for x, y in zip(x_valid[:3], y_valid[:3]):
    print(f'Label: {y} Input: "{x}"')

Loaded dataset size: x-22000, y-22000
20000 training examples.
2000 valid examples.

First 3 training examples:
Label: True Input: "Query: is a little caffeine ok during pregnancy Passage: We donât know a lot about the effects of caffeine during pregnancy on you and your baby. So itâs best to limit the amount you get each day. If youâre pregnant, limit caffeine to 200 milligrams each day. This is about the amount in 1Â½ 8-ounce cups of coffee or one 12-ounce cup of coffee."
Label: False Input: "Query: is a little caffeine ok during pregnancy Passage: It is generally safe for pregnant women to eat chocolate because studies have shown to prove certain benefits of eating chocolate during pregnancy. However, pregnant women should ensure their caffeine intake is below 200 mg per day."
Label: True Input: "Query: what fruit is native to australia Passage: Passiflora herbertiana. A rare passion fruit native to Australia. Fruits are green-skinned, white fleshed, with an unknown edible rat

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'microsoft/MiniLM-L12-H384-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/addullah/miniconda3/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/addullah/miniconda3/envs/myenv/lib/python3.12/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/home/addullah/miniconda3/envs/myenv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
from statistics import mean, stdev

lengths = [len(tokens) for tokens in tokenizer(x_train[:1_000])['input_ids']]
print(f'Mean length in tokens: {mean(lengths):0.2f}')
print(f'Stdev length in tokens: {stdev(lengths):0.2f}')

Mean length in tokens: 89.90
Stdev length in tokens: 33.08


In [11]:
# Since the mean passage length is ~90 tokens, a maximum of 250 tokens should be more than enough to acommodate
# query+passage tokens in the input.
max_length = 300
x_train_tokenized = tokenizer(x_train, max_length=max_length, truncation=True)
x_valid_tokenized = tokenizer(x_valid, max_length=max_length, truncation=True)

In [12]:
from torch.utils import data

class Dataset(data.Dataset):
    def __init__(self, examples, targets=None):
        self.examples = examples
        self.targets = targets
    
    def __len__(self):
        return len(self.examples['input_ids'])
    
    def __getitem__(self, idx):
        label = 0
        if self.targets is not None:
            label = int(self.targets[idx])
                
        return {
            'input_ids': self.examples['input_ids'][idx],
            'attention_mask': self.examples['attention_mask'][idx],
            'labels': label,
        }

dataset_train = Dataset(x_train_tokenized, y_train)
assert len(dataset_train[0]['input_ids']) > 0
assert len(dataset_train[1]['attention_mask']) > 0
assert type(dataset_train[2]['labels']) == int

In [13]:
dataset_train = Dataset(x_train_tokenized, y_train)
dataset_valid = Dataset(x_valid_tokenized, y_valid)

In [14]:
from transformers import BatchEncoding

def collate_fn(batch):
    return BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))

dataloader_train = data.DataLoader(dataset_train, batch_size=32, shuffle=True, collate_fn=collate_fn)
dataloader_valid = data.DataLoader(dataset_valid, batch_size=32, shuffle=False, collate_fn=collate_fn)

for batch in dataloader_train:
    assert batch['input_ids'].shape[0] <= dataloader_train.batch_size
    assert batch['input_ids'].shape[1] <= max_length
    break

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


# Finetuning

In [15]:
# We first define an evaluation function to measure accuracy and loss.
# It is not common to use accuracy to evaluate IR models, but it is expensive to run the reranking task in the training
# loop.

def evaluate(model, dataloader, set_name):
    losses = []
    correct = 0
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, mininterval=0.5, desc=set_name, disable=False):
            outputs = model(**batch.to(device))
            loss_val = outputs.loss
            losses.append(loss_val.cpu().item())
            preds = outputs.logits.argmax(dim=1)
            correct += (preds == batch['labels']).sum().item()

    print(f'{set_name} loss: {mean(losses):0.3f}; {set_name} accuracy: {correct / len(dataloader.dataset):0.3f}')

In [17]:
import torch
from torch import nn
from torch import optim
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
print('Parameters', model.num_parameters())

epochs = 2
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
num_training_steps = epochs * len(dataloader_train)
num_warmup_steps = int(num_training_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

# First validation to check if evaluation code is working and accuracy is random as expected 
evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')


for epoch in tqdm(range(epochs), desc='Epochs'):
    model.train()
    train_losses = []
    for batch in tqdm(dataloader_train, mininterval=0.5, desc='Train', disable=False):
        optimizer.zero_grad()
        outputs = model(**batch.to(device))
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_losses.append(loss.cpu().item())

    print(f'Epoch: {epoch + 1} Training loss: {mean(train_losses):0.2f}')
    evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameters 33360770


Valid: 100%|██████████| 63/63 [00:01<00:00, 35.33it/s]


Valid loss: 0.693; Valid accuracy: 0.500


Train: 100%|██████████| 625/625 [00:50<00:00, 12.34it/s]


...

Valid loss: 0.263; Valid accuracy: 0.893


Train: 100%|██████████| 625/625 [00:43<00:00, 14.37it/s]


Epoch: 2 Training loss: 0.17


Epochs: 100%|██████████| 2/2 [01:36<00:00, 48.48s/it]

Valid loss: 0.240; Valid accuracy: 0.911


# Evaluating on TREC-DL 2020

In [18]:
import collections


def load_queries(path):
    """Returns a dictionary whose keys are query ids and values are query texts."""
    queries = {}
    with open(path) as f:
        for line in f:
            query_id, query_text = line.strip().split('\t')
            queries[query_id] = query_text
    return queries


def load_run(path):
    """Returns a dictionary whose keys are query ids and values are lists of passage ids."""
    run = collections.defaultdict(list)
    with open(path) as f:
        for line in f:
            query_id, _, passage_id, _, _, _ = line.strip().split()
            run[query_id].append(passage_id)
    return run


def load_corpus(path):
    """Returns a dictionary whose keys are passage ids and values are passage texts."""
    corpus = {}
    with open(path) as f:
        for line in f:
            passage_id, passage_text = line.strip().split('\t')
            corpus[passage_id] = passage_text
    return corpus


queries = load_queries(path='topics.dl20.txt')
run = load_run(path='run.bm25.dl20.txt')
corpus = load_corpus(path='collection.tsv')

In [38]:
def convert_run_to_examples(run, corpus, queries):
    """Returns:
        - input_examples: a list of query-pairs pairs serialized in the same format used during finetuning.
        - query_passage_ids: a list of query-passage ids aligned with input_examples.
    """

    input_examples = []
    query_passage_ids = []
    for query_id, passage_ids in run.items():
        for passage_id in passage_ids:
            query_text = queries[query_id]
            passage_text = corpus[passage_id]
            input_text = serialize_query_passage(query_text, passage_text)
            input_examples.append(input_text)
            query_passage_ids.append((query_id, passage_id))
    return input_examples, query_passage_ids


x_test, query_passage_ids = convert_run_to_examples(run=run, corpus=corpus, queries=queries)

In [39]:
# Convert examples to batches
x_test_tokenized = tokenizer(x_test, max_length=max_length, truncation=True)
dataset_test = Dataset(x_test_tokenized, targets=None)
dataloader_test = data.DataLoader(dataset_test, batch_size=32, shuffle=False, collate_fn=collate_fn)

We now predict relevance scores for each query-passage pair

In [40]:
model.eval()
with torch.no_grad():
    scores = []
    for batch in tqdm(dataloader_test, mininterval=0.5, desc='Test', disable=False):
        outputs = model(**batch.to(device))
        scores.extend(outputs.logits[:, 1].tolist())

Test: 100%|██████████| 1688/1688 [00:38<00:00, 43.93it/s]


In [41]:
# We now convert the results to a "run" object in the format expected by pytrec_eval.
assert len(query_passage_ids) == len(scores)

run = collections.defaultdict(dict)
for (query_id, passage_id), score in zip(query_passage_ids, scores):
    run[query_id][passage_id] = score

In [42]:
len(scores), len(query_passage_ids)

(54000, 54000)

In [43]:
import pytrec_eval

with open('qrels.dl20-passage.txt', 'r') as f_qrel:
    qrels = pytrec_eval.parse_qrel(f_qrel)

metric = 'ndcg_cut_10'
evaluator = pytrec_eval.RelevanceEvaluator(qrels, {metric})
results = evaluator.evaluate(run)
print(f'nDCG@10: {sum(item[metric] for item in results.values()) / len(results):0.4f}')

nDCG@10: 0.6126


In [26]:
# qrels
# run

In [45]:
# import pytrec_eval

# with open('qrels.dl20-passage.txt', 'r') as f_qrel:
#     qrels = pytrec_eval.parse_qrel(f_qrel)

metric = 'recip_rank'
evaluator = pytrec_eval.RelevanceEvaluator(qrels, {metric})
results = evaluator.evaluate(run)
print(f'recip_rank: {sum(item[metric] for item in results.values()) / len(results):0.4f}')

recip_rank: 0.8534


In [28]:
pytrec_eval.supported_measures

{'11pt_avg',
 'G',
 'P',
 'Rndcg',
 'Rprec',
 'Rprec_mult',
 'binG',
 'bpref',
 'gm_bpref',
 'gm_map',
 'infAP',
 'iprec_at_recall',
 'map',
 'map_cut',
 'ndcg',
 'ndcg_cut',
 'ndcg_rel',
 'num_nonrel_judged_ret',
 'num_q',
 'num_rel',
 'num_rel_ret',
 'num_ret',
 'recall',
 'recip_rank',
 'relative_P',
 'relstring',
 'runid',
 'set_F',
 'set_P',
 'set_map',
 'set_recall',
 'set_relative_P',
 'success',
 'utility'}